In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_csv('scrapped_data_large.csv')
#data

In [ ]:
import missingno as msno

dataCopy = data.copy()
dataCopy = dataCopy.replace('-', np.NaN)
dataCopy = dataCopy.replace('', np.NaN)

msno.matrix(df=dataCopy.iloc[:,4:], figsize=(15, 6))

In [ ]:
dataCopy = dataCopy.fillna(0)
msno.matrix(df=dataCopy.iloc[:,4:], figsize=(15, 6))
# dataCopy

In [ ]:
dataCopy['first_class.highest'] = dataCopy['first_class.highest'].map(lambda x: str(x).strip('*'))
# dataCopy
# dataCopy.dtypes

In [ ]:
#  A
# Assuming team India to be only India and not the others like India under 21 etc
players_india_list = list(dataCopy[dataCopy.teams=='India,'].name)
players_india_list

In [ ]:
player_features = dataCopy.iloc[:,4:17]
player_classes = dataCopy.teams=='India,'
player_X = np.array(player_features)  
player_y = np.array(player_classes).T
# print('Feature set shape:', player_X.shape)
# print('Response class shape:', player_y.shape)

In [ ]:
#  B
#  Using variance threshold method for feature scaling to find the top three features that contribute to team india

from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=.15)
vt.fit(player_X, player_y)

feature_scores = [(item, score) for item, score in zip(player_features, vt.variances_)]
print("Variance threshold top 3 features, with 0 in na:\n {0}".format(sorted(feature_scores, key=lambda x: -x[1])[:3]))

In [ ]:
#  C
#  Repeating the data cleaning process for a knn imputed dataset
from missingpy import KNNImputer

knn_data = data.copy()
knn_data = knn_data.replace('-', np.NaN)
knn_data = knn_data.replace('', np.NaN)
knn_data['first_class.highest'] = knn_data['first_class.highest'].map(lambda x: str(x).strip('*'))

imputer = KNNImputer()
knn_data_imputed = imputer.fit_transform(knn_data.iloc[:,4:],5)
#knn_data_imputed

In [ ]:
knn = pd.DataFrame(knn_data_imputed[::])
cols = list(knn_data.columns[4:])
knn.columns = cols

In [ ]:
player_features = knn
player_classes = knn_data.teams=='India,'
player_X = np.array(player_features)  
player_y = np.array(player_classes).T
print('Feature set shape:', player_X.shape)
print('Response class shape:', player_y.shape)

from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=.15)
vt.fit(player_X, player_y)

feature_scores = [(item, score) for item, score in zip(player_features, vt.variances_)]
print("Variance threshold top 3 features, with knn Imputed value in na:\n {0}".format(sorted(feature_scores, key=lambda x: -x[1])[:3]))